## File type demo

This notebook demonstrates some of the capabilities of the eegyolk library in terms of reading the cnt file type. It also demonstrates the importance of the dataformat argument in the mne library. Previous work had let some arguments in reading cnt files default to preset values.
This notebook specifically demonstrates how whether the format of int32 or int16 influences the understanding of the file

#### Imports

In [4]:
import mne      # toolbox for analyzing and visualizing EEG data
import os       # using operating system dependent functionality (folders)
import pandas   # data analysis and manipulation
import numpy    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
import glob
import numpy as np
from numpy.fft import fft, fftfreq
from scipy import signal

import matplotlib
import matplotlib.pyplot as plt

from mne.time_frequency.tfr import morlet
from mne.viz import plot_filter, plot_ideal_filter


from IPython.display import clear_output
import warnings

import sys

Below eegyolk is importted in it's most updated version
But the cell could be replaced by importing the stable library and then importing the modules of it.

In [5]:

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
import helper_functions as hf # library useful for eeg and erp data cleaning
#import initialization_functions #library to import data
import epod_helper
import raw
from config import Config
from raw import RawData

#### Load EEG files
Change your config file to change where data is coming from

In [7]:
!pwd

/c/Projects/eegyolk/demos


In [6]:
config = Config()


ValueError: 
Cannot load config.

Please create a file in either one of the locations
listed below:
./config.json
C:\Users\makeda/.eegyolk/config.json
/etc/eegyolk/config.json

With the contents that specifies at least the root
directory as follows:

{
    "root": "/path/to/storage"
}

The default directory layout is expected to be:

{
    "root": "/path/to/storage",
    "data": "$root/data/",
    "preprocessed": "$root/preprocessed/",
    "metadata": "$root/metadata/",
    "models": "$root/models/"
}

You can override any individual directory by specifying it
in config.json.

"data" and "metadata" directories are expected to exist.
The "data" directory is expected to have "11mnd mmn" ..
"47mnd mmn" directories.  The "metadata" directory is
expected to have "ages" subdirectory with files named
"ages_11mnths.txt" .. "ages_47mnths.txt".

The "models" and "preprocessed" directories need not
exist.  They will be created if missing.  The "preprocessed"
directory will be used to output CSV a

In [ ]:
eeg_file_root = "../../volume-ceph/ePodium_projectfolder"
acquired = RawData(config.get_directory('data'), config.get_directory('metadata'))

Note we assume you are looking at bdf files

In [ ]:
eeg_list = []
eeg_filenames = []
eeg_filelocation = []



pattern = os.path.join(eeg_file_root, '**/*.bdf')

files = glob.glob(pattern, recursive=True)

for file in files:
    if file.endswith('.bdf'):  # and len(file) == 8:
        BdfFile = mne.io.read_raw_bdf(file)
        eeg_list.append(BdfFile)
        eeg_filenames.append(file.replace(".bdf", ""))
        eeg_filelocation.append(file)
        clear_output(wait=True)
print(len(eeg_list), "EEG files loaded")

In [ ]:
# eeg_filelocation

In [ ]:
# eeg_filenames

### Make a synthetic EEG like array

In [ ]:


sfreq = 1000. # signal collection frequency?
f_p = 40. # this is the stop freuqency, above which we should 0
flim = (1., sfreq / 2.)  # limits for plotting

In [ ]:
# make something to filter
dur = 10.
center = 2.
morlet_freq = f_p
tlim = [center - 0.2, center + 0.2]
tticks = [tlim[0], center, tlim[1]]
flim = [20, 70]

x = np.zeros(int(sfreq * dur) + 1)
blip = morlet(sfreq, [morlet_freq], n_cycles=7)[0].imag / 20.
n_onset = int(center * sfreq) - len(blip) // 2
x[n_onset:n_onset + len(blip)] += blip
x_orig = x.copy()

rng = np.random.RandomState(0)
x += rng.randn(len(x)) / 1000.
x += np.sin(2. * np.pi * 60. * np.arange(len(x)) / sfreq) / 2000.

In [ ]:
plt.plot(x)

## Download sample data from MNE library
The below is cell is commented out as it may be skipped if you have the data

In [ ]:
# sample_data_folder = mne.datasets.sample.data_path()
# sample_data_raw_file = os.path.join(sample_data_folder, 'MEG', 'sample',
#                                     'sample_audvis_raw.fif')
# raw = mne.io.read_raw_fif(sample_data_raw_file)
# # use just 60 seconds of data and mag channels, to save memory
# raw.crop(0, 60).pick_types(meg='mag', stim=True).load_data()

Let's see what filtering looks like visually if we pull out low frequencies

In [ ]:
with mne.viz._figure.use_browser_backend('matplotlib'):
        fig = raw.plot(duration=60, proj=False,
                                n_channels=len(raw.ch_names), remove_dc=False)

In [ ]:
#for cutoff in (0.1, 0.2):
cutoff = 0.2
raw_highpass = raw.copy().filter(l_freq=cutoff, h_freq=None)
with mne.viz._figure.use_browser_backend('matplotlib'):
    fig = raw_highpass.plot(duration=60, proj=False,
                            n_channels=len(raw.ch_names), remove_dc=False)
fig.subplots_adjust(top=0.9)
fig.suptitle('High-pass filtered at {} Hz'.format(cutoff), size='xx-large',
             weight='bold')

In [ ]:
raw_simple_filtered = hf.band_pass_filter(raw, 0.2, 5)

In [ ]:
## we can check channel names
#raw_simple_filtered.ch_names

In [ ]:
## we can know more about the data 
#help(raw_simple_filtered)

In [ ]:
# but let's just check our filter on the lowpass end to see what frequency is there
raw_simple_filtered.info['lowpass']

In [ ]:
# vars(raw_simple_filtered)

In [ ]:
print(type(raw.info['lowpass']))

In [ ]:
## Need to rewrite data path

In [ ]:
sample_eeg = eeg_filelocation[3]

In [ ]:
sample_eeg

In [ ]:
type(sample_eeg)

In [ ]:
sample_eeg_read = mne.io.read_raw_bdf(sample_eeg, preload= True)

In [ ]:
sample_eeg_filtered = hf.band_pass_filter(sample_eeg_read, 0, 2.38540286e-05)